<a href="https://colab.research.google.com/github/ProfessionalPallav20014/RAG-Based-AI-PDF-Summarizer/blob/main/pdf_ops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This cell installs the essential Python libraries needed for our RAG (Retrieval Augmented Generation) system. These include `langchain` for building LLM applications, `pypdf` for reading PDF files, `sentence-transformers` for creating text embeddings, `faiss-cpu` for efficient similarity search, and `transformers` for working with pre-trained models.

In [ ]:
!pip install langchain langchain-community pypdf sentence-transformers faiss-cpu transformers

This cell installs or updates `langchain-text-splitters`, a library used to break down large documents into smaller, more digestible pieces. This is important for the RAG system to process information effectively.

In [ ]:
!pip install -U langchain-text-splitters

This cell installs `langchain-classic`, which provides components for building applications with large language models, including the `RetrievalQA` chain used in this project.

In [ ]:
!pip install langchain-classic

This cell installs `Gradio`, a library that helps us create a simple and interactive web interface for our RAG system, so we can easily ask questions and get answers.

In [ ]:
# Install Gradio
!pip install gradio

This cell imports all the important tools (Python libraries) we'll need for our RAG system, like `os` for interacting with the operating system, `logging` to see what's happening behind the scenes, and various components from `langchain` and `transformers` to handle documents, embeddings, and the language model.

In [ ]:
import os
import logging
from google.colab import files
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_classic.chains.retrieval_qa.base import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

This cell defines a special blueprint called `LocalRAGSystem`. Think of it as a master plan for building our question-answering robot. It includes all the steps like uploading documents, splitting them, understanding their meaning, and finally answering questions using a local language model.

This RAG model is not like the complicated or accurate one and it's made for the purpose of running the model locally without pulling any big model with billions of parameters, which can work better but they will occupy a lot of disk space.

In [ ]:
# to keep things organized
class LocalRAGSystem:
  def __init__(self):
    self.documents=[]
    self.vector_store=None
    self.embeddings=None
    self.llm=None
    self.qa_chain=None

  # to upload the pdf dynamically in google colab
  def upload_pdfs(self):
    uploaded=files.upload()
    pdf_paths=list(uploaded.keys())
    logger.info(f"Uploaded PDFs: {pdf_paths}")
    return pdf_paths

  # to load and parse PDF documents
  def load_documents(self, pdf_paths):
      for pdf_path in pdf_paths:
          loader = PyPDFLoader(pdf_path)
          documents = loader.load()
          self.documents.extend(documents)
      logger.info(f"Loaded {len(self.documents)} pages in total.")

  def split_documents(self, chunk_size=1000, chunk_overlap=200):
      text_splitter = RecursiveCharacterTextSplitter(
          chunk_size=chunk_size, chunk_overlap=chunk_overlap)
      self.document_chunks = text_splitter.split_documents(self.documents)
      logger.info(f"Split into {len(self.document_chunks)} chunks.")

  def setup_embeddings(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
      self.embeddings = HuggingFaceEmbeddings(model_name=model_name)
      logger.info(f"Embedding model {model_name} loaded.")

  def create_vector_store(self):
      self.vector_store = FAISS.from_documents(
          self.document_chunks, self.embeddings)
      logger.info("Created the FAISS vector store.")

  def setup_local_llm(self, model_id="google/flan-t5-base", device="auto"):
      tokenizer = AutoTokenizer.from_pretrained(model_id)
      model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map=device)
      pipe = pipeline("text-generation", model=model,
                      tokenizer=tokenizer, max_new_tokens=512, temperature=0.7)
      self.llm = HuggingFacePipeline(pipeline=pipe)
      logger.info(f"Local LLM {model_id} ready.")

  # setting up the retrieval qa chain
  def setup_qa_chain(self, k=3):
      self.qa_chain = RetrievalQA.from_chain_type(
          llm=self.llm,
          chain_type="stuff",
          retriever=self.vector_store.as_retriever(search_kwargs={"k": k})
      )
      logger.info(f"Retrieval QA chain set with top {k} documents retrieved.")

  # answer questions using the RAG system
  def answer_question(self, question):
      answer = self.qa_chain.run(question)
      logger.info(f"Answered question: {question}")
      return answer

  # run the setup
  def run_setup(self, chunk_size=1000, chunk_overlap=200, model_id="google/flan-t5-base", k=3):
      pdf_paths = self.upload_pdfs()
      self.load_documents(pdf_paths)
      self.split_documents(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
      self.setup_embeddings()
      self.create_vector_store()
      self.setup_local_llm(model_id=model_id)
      self.setup_qa_chain(k=k)
      logger.info("RAG summarizer is ready to answer questions.")

This cell shows us how to use our question-answering robot! It creates an instance of our `LocalRAGSystem`, sets it up with some documents, and then asks two example questions to show how it works.

In [ ]:
# example usage
if __name__ == "__main__":
    rag = LocalRAGSystem()
    rag.run_setup()

    q1 = "What is the main topic of these documents?"
    print(f"Q: {q1}\nA: {rag.answer_question(q1)}")

    q2 = "Summarize the key points from the documents in few words and make it easy to understand."
    print(f"Q: {q2}\nA: {rag.answer_question(q2)}")

    q3 = "What is Modular RAG?"
    print(f"Q: {q3}\nA: {rag.answer_question(q3)}")

This cell defines a special function that our Gradio web interface will use. When you type a question into the web app, this function will take your question and feed it to our RAG system to get an answer.

In [ ]:
import gradio as gr

def rag_answer_question(question):
    if rag is None or rag.qa_chain is None:
        return "RAG system not initialized. Please run the setup first."
    return rag.answer_question(question)

This cell brings our question-answering robot to life as a simple website! It creates an interactive interface using Gradio, allowing you to type in questions and see the answers directly in your web browser.

In [ ]:
interface = gr.Interface(
    fn=rag_answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs=gr.Textbox(lines=15), # Increased lines for better output visibility
    title="Local RAG System Question Answering",
    description="Ask a question about the documents loaded into the RAG system."
)

# Set share=True to get a public link accessible outside your Colab session
# (This is often default behavior in Colab, but good to be explicit)
interface.launch(share=True)